## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Bubaque,GW,11.2833,-15.8333,76.50,71,4,clear sky
1,1,Vaini,TO,-21.2000,-175.2000,80.60,74,20,few clouds
2,2,Albany,US,42.6001,-73.9662,64.00,63,100,overcast clouds
3,3,Anloga,GH,5.7947,0.8973,80.60,83,20,few clouds
4,4,Tilichiki,RU,60.4667,166.1000,13.39,78,100,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) &\
                                       (city_data_df["Max Temp"]>=min_temp)]
vacation_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Current Description
0,0,Bubaque,GW,11.2833,-15.8333,76.50,71,4,clear sky
1,1,Vaini,TO,-21.2000,-175.2000,80.60,74,20,few clouds
3,3,Anloga,GH,5.7947,0.8973,80.60,83,20,few clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.57,75,14,few clouds
6,6,Nuevitas,CU,21.5453,-77.2644,73.40,83,20,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.notnull().sum()

City_ID                238
City                   238
Country                236
Lat                    238
Lng                    238
Max Temp               238
Humidity               238
Cloudiness             238
Current Description    238
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Vacation_cities_df = vacation_cities_df.dropna()
Vacation_cities_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Current Description    236
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   236
Country                236
Max Temp               236
Current Description    236
Lat                    236
Lng                    236
Hotel Name             236
dtype: int64

In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
vacation_hotels_df = hotel_df.dropna()
vacation_hotels_df.count()

City                   236
Country                236
Max Temp               236
Current Description    236
Lat                    236
Lng                    236
Hotel Name             236
dtype: int64

In [30]:
# pandas is not reading the empty hotels values, so I exported the data into a csv file & imported
# it back to drop null values
output_data_file = "WeatherPy_hotels.csv"
vacation_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
clean_hotel_df = pd.read_csv("WeatherPy_hotels.csv")
clean_hotel_df.isnull().sum()

City_ID                 0
City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             17
dtype: int64

In [34]:
clean_hotels_df = clean_hotel_df.dropna()
clean_hotels_df.count()

City_ID                219
City                   219
Country                219
Max Temp               219
Current Description    219
Lat                    219
Lng                    219
Hotel Name             219
dtype: int64

In [36]:
# 8 create a csv file with the output 
output_data_file = "WeatherPy_vacation.csv"
clean_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
clean_hotels_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Bubaque,GW,76.50,clear sky,11.2833,-15.8333,"Casa Dora, Bubaque"
1,1,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort
2,3,Anloga,GH,80.60,few clouds,5.7947,0.8973,Pin Drop Hotel
3,5,Rikitea,PF,78.57,few clouds,-23.1203,-134.9692,Pension Maro'i
4,6,Nuevitas,CU,73.40,few clouds,21.5453,-77.2644,Hotel Caonaba


In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country code</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Current Description} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotels_df[["Lat", "Lng"]]
# Add a heatmap of temperature for the vacation spots.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11a. Add a marker layer for each city to the map. 
# 11b. Display the figure.
fig

Figure(layout=FigureLayout(height='420px'))